In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [2]:
BUFFER_SIZE = 70_000
BATCH_SIZE = 128

NUM_EPOCHS = 20

In [3]:
mnist_dataset, mnist_info = tfds.load(name='mnist',with_info=True, as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\szizo\tensorflow_datasets\mnist\3.0.1.incompleteXW5S1P\mnist-train.tfrecord*...:   0%|     …

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\szizo\tensorflow_datasets\mnist\3.0.1.incompleteXW5S1P\mnist-test.tfrecord*...:   0%|      …

Dataset mnist downloaded and prepared to C:\Users\szizo\tensorflow_datasets\mnist\3.0.1. Subsequent calls will reuse this data.


In [4]:
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

In [5]:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255
    
    return image, label

In [7]:
train_and_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

In [8]:
num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

In [9]:
num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

In [10]:
train_and_validation_data = train_and_validation_data.shuffle(BUFFER_SIZE)

In [11]:
train_data = train_and_validation_data.skip(num_validation_samples)
validation_data = train_and_validation_data.take(num_validation_samples)

In [12]:
train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

In [14]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(50,5,activation='relu', input_shape=(28,28,1)),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Conv2D(50,3,activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10)
])

In [15]:
model.summary(line_length = 75)

Model: "sequential"
___________________________________________________________________________
 Layer (type)                    Output Shape                  Param #     
 conv2d (Conv2D)                 (None, 24, 24, 50)            1300        
                                                                           
 max_pooling2d (MaxPooling2D)    (None, 12, 12, 50)            0           
                                                                           
 conv2d_1 (Conv2D)               (None, 10, 10, 50)            22550       
                                                                           
 max_pooling2d_1 (MaxPooling2D)  (None, 5, 5, 50)              0           
                                                                           
 flatten (Flatten)               (None, 1250)                  0           
                                                                           
 dense (Dense)                   (None, 10)                    12510

In [16]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True)

In [17]:
model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])

In [18]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor = 'val_loss',
    mode = 'auto',
    min_delta = 0,
    patience = 2,
    verbose = 0,
    restore_best_weights = True
)

In [19]:
model.fit(
    train_data,
    epochs = NUM_EPOCHS,
    callbacks = [early_stopping],
    validation_data = validation_data,
    verbose = 2
)

Epoch 1/20
422/422 - 59s - loss: 0.2668 - accuracy: 0.9231 - val_loss: 0.0767 - val_accuracy: 0.9773 - 59s/epoch - 139ms/step
Epoch 2/20
422/422 - 46s - loss: 0.0689 - accuracy: 0.9794 - val_loss: 0.0509 - val_accuracy: 0.9835 - 46s/epoch - 110ms/step
Epoch 3/20
422/422 - 57s - loss: 0.0533 - accuracy: 0.9835 - val_loss: 0.0506 - val_accuracy: 0.9843 - 57s/epoch - 136ms/step
Epoch 4/20
422/422 - 54s - loss: 0.0433 - accuracy: 0.9866 - val_loss: 0.0318 - val_accuracy: 0.9915 - 54s/epoch - 128ms/step
Epoch 5/20
422/422 - 55s - loss: 0.0364 - accuracy: 0.9890 - val_loss: 0.0302 - val_accuracy: 0.9898 - 55s/epoch - 131ms/step
Epoch 6/20
422/422 - 48s - loss: 0.0304 - accuracy: 0.9905 - val_loss: 0.0223 - val_accuracy: 0.9943 - 48s/epoch - 113ms/step
Epoch 7/20
422/422 - 45s - loss: 0.0271 - accuracy: 0.9915 - val_loss: 0.0172 - val_accuracy: 0.9947 - 45s/epoch - 106ms/step
Epoch 8/20
422/422 - 45s - loss: 0.0234 - accuracy: 0.9926 - val_loss: 0.0202 - val_accuracy: 0.9935 - 45s/epoch - 106

In [20]:
test_loss, test_accuracy = model.evaluate(test_data)

1/1 [==============================] - 1s 1s/step - loss: 0.0327 - accuracy: 0.9901
